### Installation

In [38]:
# !pip install folium

### Imports

In [39]:
import folium
import numpy as np
import pandas as pd
from folium import plugins
from folium.features import CustomIcon, FeatureGroup

### Read data from "xlsx" file

In [40]:
testimonios = pd.read_excel("../data/processed/testimonios_cuba.xlsx", index_col=None, sheet_name="base_limpia", converters={"ID": str})

testimonios.shape

(197, 5)

### Dataframe sample

In [41]:
testimonios.sample(1)

,Testimonio,Provincia,Año,Hospital,ID
104,\nPagué por tener a un doctor pendiente a mi e...,Matanzas,2017,Hospital Provincial Ginecobstétrico Julio Rafa...,382


### Explore data

In [42]:
# Number of unique values from column
print(testimonios['Provincia'].nunique())

# Unique values for a column to a list
provincias = list(testimonios['Provincia'].dropna().unique())
provincias

14


['La Habana',
 'Pinar del Río',
 'Mayabeque',
 'Camagüey ',
 'Isla de la Juventud',
 'Matanzas',
 'Sancti Spíritus',
 'Villa Clara',
 'Ciego de Ávila',
 'Holguín',
 'Artemisa ',
 'Las Tunas',
 'Santiago de Cuba',
 'Cienfuegos']

### Filter data

In [43]:
# Function to filter data according to column values
def filter_prov(df, col="Name", val=list):
    dfs = []
    for i in range(len(provincias)):
        prov = df.loc[df[col] == val[i]]
        dfs.append(prov)
    return list(dfs)

p1, p2, p3, p4, p5, p6, p7, p8, p9, p10, p11, p12, p13, p14 = filter_prov(testimonios, col="Provincia", val=provincias)

### Create random coordinates for each dataframe

In [106]:
coordinates = [
    [23.03353, 23.09987, -82.15752, -82.50496], # La Habana
    [22.42376, 22.42995, -83.69975, -83.68952], # Pinar del Río
    [22.85195, 22.84857, -81.93923, -81.89057], # Mayabeque
    [21.37445, 21.38787, -77.90526, -77.90732], # Camagüey
    [21.68043, 21.71871, -82.79392, -82.85159], # Isla de la Juventud
    [23.02835, 23.02646, -81.54440, -81.56225], # Matanzas
    [21.94163, 21.93876, -79.43991, -79.44849], # Sancti Spiritus
    [22.56186, 22.45530, -79.94375, -79.97671], # Villa Clara
    [21.84588, 21.84460, -78.75283, -78.75678], # Ciego de Ávila
    [20.88537, 20.88825, -76.25460, -76.26147], # Holguín
    [22.81586, 22.81665, -8275735, -82.75941],  # Artemisa
    [20.96658, 20.96722, -76.95149, -76.95801], # Las Tunas
    [20.02077, 20.02884, -75.81065, -75.81374], # Santiago de Cuba
    [22.16815, 22.15956, -80.42153, -80.42599]  # Cienfuegos
]

"""
Function to create random coordinates
according to dataframes rows length and
within a range of bounding coordinates
for each province in Cuba
"""
def create_coordinates(*args, coor=list):
    dfs = list(args)
    for i in range(len(dfs)):
        x = list(np.random.uniform(coor[i][0], coor[i][1], len(dfs[i])))
        y = list(np.random.uniform(coor[i][2], coor[i][3], len(dfs[i])))
        dfs[i] = dfs[i].assign(Long = x)
        dfs[i] = dfs[i].assign(Lat = y)
    return list(dfs)

p1, p2, p3, p4, p5, p6, p7, p8, p9, p10, p11, p12, p13, p14 = create_coordinates(p1, p2, p3, p4, p5, p6, p7, p8, p9, p10, p11, p12, p13, p14, coor=coordinates)

### Concatenate dataframes

In [107]:
# Create a list with dataframes
provincias = [p1, p2, p3, p4, p5, p6, p7, p8, p9, p10, p11, p12, p13, p14]

# Concatenate the dataframes
df_join = pd.concat(provincias, ignore_index=True)

df_join.shape

(196, 7)

### Create a map

In [116]:
# Create a map at specified location
m = folium.Map(location=[22.037635528256533, -79.36879425670136], zoom_start=7, 
               tiles=None)

# Add layer and set dark as default mode
folium.TileLayer(tiles="Stamen Watercolor", name="Testimonios", attr= '''
               &copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> 
               contributors &copy; <a href="https://carto.com/attributions">CARTO</a>
               ''').add_to(m)
# Add fullscreen option
plugins.Fullscreen(
    position="topright",
    title="Pantalla completa",
    title_cancel="Salir de pantalla completa",
    force_separate_button=True,
).add_to(m)

# Add a group
testimonios = FeatureGroup(name="Testimonios")
testimonios.add_to(m)

# Custom icon image
icon_image = "../assets/icon.png"


# Add markers
for i, r in df_join.iterrows():

    icon = CustomIcon(icon_image, 
    icon_size=(50, 50),
    icon_anchor=(20, 10),
    popup_anchor=(-3, -20))

    folium.Marker([r['Long'], r['Lat']],
    popup = folium.Popup('''
                        <div class='text'>
                            <p>{}</p>
                        </div>

                        <style type='text/css'>
                            .text {{
                                font-family: Verdana, sans-serif;
                                text-align:justify;
                                font-size:12px;
                                font-weight: 600; 
                                width: 500px;
                                border: black solid 2px;
                                # border-radius: 5px;
                                padding: 20px;
                                box-shadow: 0 -5px 5px -3px black, 0 5px 5px -3px black;
                            }}

                            .leaflet-popup-content-wrapper, .leaflet-popup-tip {{
                                background-image: radial-gradient( circle farthest-corner at 52.1% -29.6%,  rgba(144,17,105,1) 0%, rgba(51,0,131,1) 100.2% );
                            }}

                            .leaflet-popup-content {{
                                background-color: #85FFBD;
                                background-image: linear-gradient(45deg, #85FFBD 0%, #FFFB7D 100%);
                                color:black;
                                }}

                            .leaflet-popup-content:hover {{
                                background-image: radial-gradient( circle farthest-corner at 7.5% 24%,  rgba(237,161,193,1) 0%, rgba(250,178,172,1) 25.5%, rgba(190,228,210,1) 62.3%, rgba(215,248,247,1) 93.8% );
                                color:black;
                                }}
                        </style>

                         '''.format(r['Testimonio'])),
    # popup= folium.Popup(iframe, min_width=300, max_width=300),
    tooltip = r['Provincia'],
    # icon = folium.Icon(color='darkpurple', icon='comment', prefix='fa')).add_to(testimonios)
    icon = icon).add_to(testimonios)

# Save map as 'html'
# m.save("../outputs/figures/mapa_testimonios.html")

m